In [0]:
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path="/content/drive/My Drive/Hackathon/amazonreviews.tsv"
rev=pd.read_csv(path,sep='\t')

In [4]:
rev.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [0]:
rev.shape

(10000, 2)

In [5]:
rev['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [6]:
rev.isnull().values.any()

False

In [0]:
blank_rows=[]
for i,lb,rv in rev.itertuples():
  if type(rv)==str:
    if rv.isspace():
      blank_rows.append(i)

In [9]:
blank_rows

[]

In [11]:
nlp=spacy.load('en_core_web_sm')
d=nlp("45678 data is waste")
d

45678 data is waste

In [16]:
#for i in d:
  #if type(i.text)==str:
    #print(i.lemma_)

45678
datum
be
waste


In [0]:
def clean_text(token):
  if token.is_stop==False and token.is_punct==False and not token.is_space:
    return True
  return False

def lemma_text(text):
    cleaned_text=[token.lemma_ for token in nlp(text) if clean_text(token)]
    return cleaned_text

In [0]:
#tfidf=TfidfVectorizer()
#rev_tfidf=tfidf
rev_cv=CountVectorizer(analyzer=lemma_text,max_features=1000)
x=rev_cv.fit_transform(rev['review'])

In [0]:
#rev_cv.get_feature_names()

In [0]:
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(x)

In [0]:
y=rev['label']

In [0]:
x_train,x_test,y_train,y_test=train_test_split(X_train_tfidf,y,test_size=0.20,random_state=0)

In [32]:
x_train.shape

(8000, 1000)

In [0]:
revnb=MultinomialNB()

In [34]:
revnb.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
y_predict=revnb.predict(x_test)
print(y_predict)

['neg' 'pos' 'neg' ... 'pos' 'neg' 'pos']


In [36]:
np.mean(y_predict == y_test)

0.8095

In [37]:
from sklearn import metrics
metrics.confusion_matrix(y_test,y_predict)

array([[824, 173],
       [208, 795]])

In [39]:
print(metrics.classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         neg       0.80      0.83      0.81       997
         pos       0.82      0.79      0.81      1003

    accuracy                           0.81      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.81      0.81      0.81      2000

